In [ ]:
%pip install --upgrade --quiet  langchain langchain-community langchain-openai langchain-ollama langchain-experimental neo4j tiktoken yfiles_jupyter_graphs python-dotenv
%pip install neo4j
%pip install json-repair
%pip install py2neo
%pip install tree_sitter
%pip install --upgrade langchain
%pip install llama-index --upgrade
%pip show llama-index
%pip install yfiles-jupyter-graphs
%pip install --upgrade yfiles-jupyter-graphs
%pip install --upgrade jupyterlab ipywidgets
%pip install jupyterlab
%pip install sqlfluff

In [33]:
%pip install langgraph

Note: you may need to restart the kernel to use updated packages.


In [13]:
from langchain_core.runnables import  RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.output_parsers import StrOutputParser
import os
from langchain_community.graphs import Neo4jGraph
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import ChatOpenAI
from langchain_community.chat_models import ChatOllama
from langchain_experimental.graph_transformers import LLMGraphTransformer
from neo4j import GraphDatabase
from yfiles_jupyter_graphs import GraphWidget
from langchain_community.vectorstores import Neo4jVector
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores.neo4j_vector import remove_lucene_chars
import re
from langchain_core.prompts import PromptTemplate
from langchain.chains import LLMChain

from dotenv import load_dotenv

load_dotenv()

True

In [7]:
uri = os.environ["NEO4J_URI"] 
username = os.environ["NEO4J_USERNAME"]
password = os.environ["NEO4J_PASSWORD"]
api_url = os.getenv('API_URL')

# Agora você pode criar o gráfico sem passar os parâmetros diretamente
graph = Neo4jGraph()

In [8]:
from langchain_openai import ChatOpenAI
from neo4j import GraphDatabase

# Função para executar a consulta Cypher no Neo4j
def execute_cypher_query(query, uri, username, password):
    driver = GraphDatabase.driver(uri, auth=(username, password))
    with driver.session() as session:
        result = session.run(query)
        # Extrair os resultados da query
        return [record.data() for record in result]

# Definir o template para geração de Cypher
CYPHER_GENERATION_TEMPLATE = """Objetivo: Gerar uma instrução Cypher para consultar o banco de dados Neo4J que possui uma estrutura de grafos representando um Schema de Banco de dados SQL.
Instruções:
- Verifique o esquema do banco de dados SQL consultando o Neo4j para obter os nomes das tabelas (nós) e colunas (propriedades).
- Converta qualquer termo no plural para o singular antes de gerar a query.
- Construa uma query Cypher para obter todas as Tabelas e Colunas que relacionem com o Prompt do usuário.
- Certifique-se de verificar os relacionamentos (chaves estrangeiras) no banco de dados gráfico.
- **Não inclua explicações ou desculpas em suas respostas.**
- **Nunca responda com "Não sei a resposta" ou frases similares.** Retorne apenas o resultado da query gerada ou uma lista vazia se não houver correspondências.
----------
Exemplo de Query Cypher para ser utilizado:
MATCH (t:Table)-[:HAS_COLUMN]->(c:Column)
WHERE t.id CONTAINS 'usuario' OR c.id CONTAINS 'usuario' OR c.id CONTAINS 'ativo' OR c.id CONTAINS 'sistema'
RETURN t.id, c.id 
----------
Relação do nome das tabelas existentes dentro do banco de dados Neo4j:
Controledeacessoapitokens, 
Controledeacessousuario, 
Empresa, 
Controledeacessogrupo, 
Controledeacessogrupousuario, 
Controledeacessomenu, 
Controledeacessomodulotela, 
Controledeacessomodulo, 
Controledeacessotela, 
Controledeacessopermissao, 
Controledeacessorecurso, 
Controledeacessopermissaomenu, 
Usuario, 
Controledeacessousuariocliente
----------
Schema banco de dados:
{schema}
----------
User prompt:
{query}
"""

# Criar o template para o prompt de Cypher
schema_data = graph.schema   
query = "Quantos usuário ativo existem no sistema?"

# Substituir as variáveis no template com o schema e query do usuário
prompt = CYPHER_GENERATION_TEMPLATE.format(schema=schema_data, query=query)

# Conectar ao modelo da OpenAI
llm = ChatOpenAI(temperature=0)

# Executar o LLM diretamente para gerar a consulta Cypher
cypher_query = llm.predict(prompt)

# Exibir a consulta Cypher gerada
print(f"Generated Cypher Query: {cypher_query}")

# Executar a consulta Cypher no Neo4j e obter os resultados
output = execute_cypher_query(cypher_query, uri, username, password)

# Exibir o resultado da execução da query Cypher
print(f"Cypher Query Output: {output}")

Generated Cypher Query: MATCH (t:Table)-[:HAS_COLUMN]->(c:Column)
WHERE toLower(t.id) CONTAINS 'usuario' OR toLower(c.id) CONTAINS 'usuario' OR toLower(c.id) CONTAINS 'ativo' OR toLower(c.id) CONTAINS 'sistema'
RETURN t.id, c.id
Cypher Query Output: [{'t.id': 'Controledeacessoapitokens', 'c.id': 'Id_Usuario'}, {'t.id': 'Controledeacessousuario', 'c.id': 'Redefinirsenha'}, {'t.id': 'Controledeacessousuario', 'c.id': 'Datadaultimaatualizacao'}, {'t.id': 'Controledeacessousuario', 'c.id': 'Databloqueiousuario'}, {'t.id': 'Controledeacessousuario', 'c.id': 'Email'}, {'t.id': 'Controledeacessousuario', 'c.id': 'Datadecriacao'}, {'t.id': 'Controledeacessousuario', 'c.id': 'Status'}, {'t.id': 'Controledeacessousuario', 'c.id': 'Tipo'}, {'t.id': 'Controledeacessousuario', 'c.id': 'Nomecompleto'}, {'t.id': 'Controledeacessousuario', 'c.id': 'Identificacao'}, {'t.id': 'Controledeacessousuario', 'c.id': 'Senha'}, {'t.id': 'Controledeacessousuario', 'c.id': 'Login'}, {'t.id': 'Controledeacessousua

In [14]:
# Definir o template para geração de SQL com uma condição de filtragem.
# trocar por tabela e coluna, inserir o prompt da smart go. melhorar esse segundo prompt.
# carregar os demais arquivos. indices ser keys, keys ser primer keys. colunas e tabelas ser nós. 
# trazer junto a coluna o titulo campo, propriedade do nó de coluna, podendo ser o tipo(type) dele.
# forest key relaciona a coluna de outra tabela. deveria ser relação.   
# mandar pro chatgpt para me ajudar a orientar sobre as create tables. 

# 2. a api rodar. substituir os metodos que a gente fez antes por esse. 
SQL_TEMPLATE = """Objetivo: Responda sempre com uma query SQL que reflita a pergunta solicitada, sem usar o caractere '*' na construção das queries.

Contexto:
- `t.id` representa o nome da tabela. Por exemplo, se `t.id` for 'Controledeacessousuario', isso significa que o nome da tabela é 'Controledeacessousuario'.
- `c.id` representa o nome da coluna. Por exemplo, se `c.id` for 'Status', isso significa que a coluna que você deve usar na query SQL é 'Status'.

Instruções:
- Use somente o nome da tabela e o nome da coluna fornecidos no `Full Context` abaixo para construir a query SQL.
- Se houver uma coluna `Status`, inclua uma condição `WHERE` que filtre usuários com o `Status = 'ativo'`.
- Não permita retornos ou queries de 'SELECT *' de uma tabela.

Full Context: {result}
----------
User prompt:
{query}
"""

# Criar o template para o prompt de SQL
SQL_GENERATION_PROMPT = PromptTemplate(
    input_variables=["result", "query"], template=SQL_TEMPLATE
)

# Agora, use o Full Context para gerar uma query SQL correspondente
sql_chain = LLMChain(
    llm=llm,
    prompt=SQL_GENERATION_PROMPT,
    verbose=True
)

# Executar a cadeia para gerar a query SQL
sql_result = sql_chain.run({
    "result": output, 
    "query": query
})

# Imprimir a consulta SQL gerada
print(f"SQL gerada: {sql_result}")

/var/folders/gf/j9d8qz2x4cz15cmbk3g15kdm0000gn/T/ipykernel_45669/3663280512.py:32: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  sql_chain = LLMChain(
/var/folders/gf/j9d8qz2x4cz15cmbk3g15kdm0000gn/T/ipykernel_45669/3663280512.py:39: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  sql_result = sql_chain.run({




> Entering new LLMChain chain...
Prompt after formatting:
Objetivo: Responda sempre com uma query SQL que reflita a pergunta solicitada, sem usar o caractere '*' na construção das queries.

Contexto:
- `t.id` representa o nome da tabela. Por exemplo, se `t.id` for 'Controledeacessousuario', isso significa que o nome da tabela é 'Controledeacessousuario'.
- `c.id` representa o nome da coluna. Por exemplo, se `c.id` for 'Status', isso significa que a coluna que você deve usar na query SQL é 'Status'.

Instruções:
- Use somente o nome da tabela e o nome da coluna fornecidos no `Full Context` abaixo para construir a query SQL.
- Se houver uma coluna `Status`, inclua uma condição `WHERE` que filtre usuários com o `Status = 'ativo'`.
- Não permita retornos ou queries de 'SELECT *' de uma tabela.

Full Context: [{'t.id': 'Controledeacessoapitokens', 'c.id': 'Id_Usuario'}, {'t.id': 'Controledeacessousuario', 'c.id': 'Redefinirsenha'}, {'t.id': 'Controledeacessousuario', 'c.id': 'Datadaultima